In [35]:
import json
import pandas as pd
import urllib.request
import time
from tqdm import tqdm
import numpy as np
import os

# Connect to Reddit using PRAW

In [4]:
import praw
reddit = praw.Reddit(client_id='b-rGm7WdyYgAUw',
                     client_secret='NTUUgRLnUso2C-y8GQkdyWykmeo',
                     user_agent='ToastMeBot')
print("Connected to reddit:", reddit.read_only) 

Connected to reddit: True


# Download the titles, comments, and URLs

In [ ]:
data = []

# Get the top 1000 posts
print("TOP")
for i, submission in tqdm(enumerate(reddit.subreddit('toastme').top(limit=1000)), total=1000):
    curr = {}
    curr['title'] = submission.title
    curr['url'] = submission.url
    curr['comments'] = [comment.body for comment in submission.comments if hasattr(comment, 'body')]
    data.append(curr)
json.dump(data, open('data.json', 'w'))

# Get the top 1000 hottest posts
print("HOT")
for i, submission in tqdm(enumerate(reddit.subreddit('toastme').hot(limit=1000)), total=1000):
    curr = {}
    curr['title'] = submission.title
    curr['url'] = submission.url
    curr['comments'] = [comment.body for comment in submission.comments if hasattr(comment, 'body')]
    data.append(curr)
json.dump(data, open('data.json', 'w'))

# Get the top 1000 newest posts
print("NEW")
for i, submission in tqdm(enumerate(reddit.subreddit('toastme').new(limit=1000)), total=1000):
    curr = {}
    curr['title'] = submission.title
    curr['url'] = submission.url
    curr['comments'] = [comment.body for comment in submission.comments if hasattr(comment, 'body')]
    data.append(curr)
json.dump(data, open('data.json', 'w'))

# Get the top 1000 most controversial posts
print("CONTROLVERSIAL")
for i, submission in tqdm(enumerate(reddit.subreddit('toastme').controversial(limit=1000)), total=1000):
    curr = {}
    curr['title'] = submission.title
    curr['url'] = submission.url
    curr['comments'] = [comment.body for comment in submission.comments if hasattr(comment, 'body')]
    data.append(curr)
json.dump(data, open('data.json', 'w'))

# Download the Reddit images

In [11]:
image_dir = "./images"
if not os.path.exists(image_dir):
    os.makedirs(image_dir)

In [ ]:
data = pd.read_json("data.json")
data["ID"] = [1 + i for i in range(len(data))]
count = 1
for img, ids in tqdm(zip(data.url, data.ID)):
    time.sleep(0.3)
    img_name = "images/ID="+ str(ids)+"_Name=" +img.split('/', 3)[3]+".jpg"
    img_name = img_name.replace(".jpg.jpg", ".jpg")
    try:
        urllib.request.urlretrieve(img, img_name)
    except Exception:
        pass
    count = count + 1

# Extract Visual Features

In [21]:
from keras.applications.vgg19 import VGG19
from keras.preprocessing import image
from keras.applications.vgg19 import preprocess_input
from keras.models import Model
import glob

In [17]:
# the vgg 19 model is used for getting the image features
vgg19_model = VGG19(weights='imagenet')
vgg19_model = Model(inputs=vgg19_model.input, outputs=vgg19_model.get_layer('predictions').output)

In [ ]:
#get visual features
image_files = glob.glob(image_dir+"/*")
img_features = []

for imgf in tqdm(image_files):
    img_path = imgf
    try:
        img = image.load_img(img_path, target_size=(224, 224))
        x = image.img_to_array(img)
        x = np.expand_dims(x, axis=0)
        x = preprocess_input(x)
        prediction_features = vgg19_model.predict(x)
        img_features.append((imgf, prediction_features.ravel()))
    except Exception as e:
        pass

In [44]:
reddit = pd.read_json("data.json")

def extract_id(text):
    s = text
    start = s.find(image_dir + '/ID=') +13
    end = s.find('_Name', start)
    return s[start:end]

image_features_dataframe = pd.DataFrame(columns = ["img_name", "image_feature"], data = img_features)
image_features_dataframe["ID"] = [int(extract_id(x)) for x in image_features_dataframe.img_name]

reddit["ID"] = [1 + i for i in range(len(reddit))]
reddit = reddit.merge(image_features_dataframe, on = 'ID')

# Save the data to be used by ToastBot for training

In [46]:
reddit.to_json("data_with_image_features.json")